In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:


import numpy as np
import pandas as pd
import math
df = pd.read_csv('train_0_tail_10k.csv')

df.pop('boost0_timer')
df.pop('boost1_timer')
df.pop('boost2_timer')
df.pop('boost3_timer')
df.pop('boost4_timer')
df.pop('boost5_timer')

df.pop('p0_boost')
df.pop('p1_boost')
df.pop('p2_boost')
df.pop('p3_boost')
df.pop('p4_boost')
df.pop('p5_boost')

df.pop('p0_pos_z')
df.pop('p1_pos_z')
df.pop('p2_pos_z')
df.pop('p3_pos_z')
df.pop('p4_pos_z')
df.pop('p5_pos_z')

df.pop('p0_vel_z')
df.pop('p1_vel_z')
df.pop('p2_vel_z')
df.pop('p3_vel_z')
df.pop('p4_vel_z')
df.pop('p5_vel_z')

df.pop('ball_pos_z')
df.pop('ball_vel_z')

df.tail()



,game_num,event_id,event_time,ball_pos_x,ball_pos_y,ball_vel_x,ball_vel_y,p0_pos_x,p0_pos_y,p0_vel_x,...,p4_vel_x,p4_vel_y,p5_pos_x,p5_pos_y,p5_vel_x,p5_vel_y,player_scoring_next,team_scoring_next,team_A_scoring_within_10sec,team_B_scoring_within_10sec
9995,737,737005,-0.473386,11.6650,25.500801,-11.0376,48.485400,36.1764,40.2512,5.5962,...,-0.2856,0.8790,-19.553400,102.059600,10.821200,-0.0028,-1,NaN,0,0
9996,737,737005,-0.363688,10.3796,31.147000,-10.9984,48.311802,36.6466,38.1190,5.6754,...,-0.3604,1.0516,-17.852600,101.806200,9.924601,-6.2408,-1,NaN,0,0
9997,737,737005,-0.254574,9.1902,36.371002,-10.9620,48.150600,37.1824,35.7382,5.9694,...,-4.7684,11.2826,-16.913599,101.149600,10.650000,-7.6132,-1,NaN,0,0
9998,737,737005,-0.145479,8.1868,40.778202,-10.9312,48.016200,37.6878,33.5450,6.1292,...,-4.7726,11.2928,-15.999201,100.512800,11.185200,-7.6824,-1,NaN,0,0
9999,737,737005,-0.036366,7.0956,45.572200,-10.8976,47.869800,38.4584,29.2822,2.5992,...,-4.7726,11.2928,-15.150000,99.929405,11.398600,-7.8708,-1,NaN,0,0


In [ ]:
# event x_a1 - ball closer to team A goal
# event x_a2 - ball closer to team B goal
# event x_b1 - ball colliding with team A player
# event x_b2 - ball colliding with team B player
# event x_c1 - ball rolling toward team A goal
# event x_c2 - ball rolling toward team B goal
import math
import sys

#how many events are in this game?
unique_kickoffs = df['event_id'].unique()
print("unique kickoffs is={}".format(unique_kickoffs))

#how many snapshots are in each event?
event_counts = df.event_id.value_counts()
print("event_counts is=\n{}".format(event_counts))


event_tables = []

#for each kickoff, let's consider our three events in an event table
for i in range(len(unique_kickoffs)):
  event_table=pd.DataFrame(columns=["game_num", "event_id", "ball_loc", "x_a1", "x_a2", "x_b1", "x_b2", "x_c1", "x_c2", "team_scoring_in_10s"])
  if(i == 0):
    start = 0
  else:
    start += event_counts[unique_kickoffs[i-1]]
  #for all snapshots in the event, we need to determine if any x_a,x_b,or x_c occurring
  for j in range(start, start+event_counts[unique_kickoffs[i]]-1):
    

    #look through original dataframe
    
    df_row = []
    df_row_check = []
    
    df_row.insert(0,df['game_num'][j])
    df_row_check.insert(0,'game_num')

    df_row.insert(1,df['event_id'][j])
    df_row_check.insert(1,'event_id')

    #determine which side of field the ball is on
    if(df['ball_pos_x'][j] < 0): 
      df_row.insert(2,'A')
      df_row_check.insert(2,'ball_loc')

    if(df['ball_pos_x'][j] > 0): 
      df_row.insert(2,'B')
      df_row_check.insert(2,'ball_loc')

    if(df['ball_pos_x'][j] == 0): 
      df_row.insert(2,math.nan)
      df_row_check.insert(2,'ball_loc')

    min_ball_goal_dist = 5
    #is ball close to team A goal
      ##assume team A goal at x = -80 
    d_ball_goal_a = math.sqrt((df['ball_pos_x'][j]+80)**2 + df['ball_pos_y'][j]**2)

    
    
    if(d_ball_goal_a <= min_ball_goal_dist):
      df_row.insert(3,'yes')
      df_row_check.insert(3,'x_a1')

    if(d_ball_goal_a > min_ball_goal_dist):
      df_row.insert(3,'no')
      df_row_check.insert(3,'x_a1')


    #is ball close to team B goal
      ##assume team B goal at x = 80
    d_ball_goal_b = math.sqrt((df['ball_pos_x'][j]-80)**2 + df['ball_pos_y'][j]**2)
    
    if(d_ball_goal_b <= min_ball_goal_dist):
      df_row.insert(4,'yes')
      df_row_check.insert(4,'x_a2')

    if(d_ball_goal_b > min_ball_goal_dist):
      df_row.insert(4,'no')
      df_row_check.insert(4,'x_a2')




    #is ball colliding with player

    #nearest_player_idx = index of nearest player
    ##need to compute nearest player
    
    min_pbd = sys.maxsize
    min_pbi = -1
    for n in range(0,6):
      player_str = 'p{ind}'.format(ind = n)
      player_str_x = player_str +  '_pos_x'
      player_str_y = player_str +  '_pos_y'
      currdist = math.sqrt((df['ball_pos_x'][j] - df[player_str_x][j])**2 + (df['ball_pos_y'][j]-df[player_str_y][j])**2)
      if(currdist < min_pbd):
        min_pbd = currdist
        min_pbi = n

    nearest_player_idx = min_pbi
    d_player_ball = min_pbd

    min_ball_player_dist = 5

    player_on_team_a = (nearest_player_idx == 0) or (nearest_player_idx == 1) or (nearest_player_idx == 2)

    player_on_team_b = (nearest_player_idx == 3) or (nearest_player_idx == 4) or (nearest_player_idx == 5)


    #is ball colliding with team A player
    if((d_player_ball <= min_ball_player_dist) and player_on_team_a):
      df_row.insert(5,'yes')
      df_row_check.insert(5,'x_b1')
      df_row.insert(6,'no')
      df_row_check.insert(6,'x_b2')



    #is ball colliding with team B player
    if((d_player_ball <= min_ball_player_dist) and player_on_team_b):
      df_row.insert(5,'no')
      df_row_check.insert(5,'x_b1')
      df_row.insert(6,'yes')
      df_row_check.insert(6,'x_b2')


    #no player is currently colliding with ball
    if(d_player_ball > min_ball_player_dist):
      df_row.insert(5,'no')
      df_row_check.insert(5,'x_b1')
      df_row.insert(6,'no')
      df_row_check.insert(6,'x_b2')


    


    #is ball rolling toward player A goal
    if(df['ball_vel_x'][j] < 0):
       df_row.insert(7,'yes')
       df_row_check.insert(7,'x_c1')
       df_row.insert(8,'no')
       df_row_check.insert(8,'x_c2')


    #is ball rolling toward player B goal
    if(df['ball_vel_x'][j] > 0):
       df_row.insert(7,'no')
       df_row_check.insert(7,'x_c2')
       df_row.insert(8,'yes')
       df_row_check.insert(8,'x_c2')


    #ball is not moving
    if(df['ball_vel_x'][j] == 0):
       df_row.insert(7,'no')
       df_row_check.insert(7,'x_c1')
       df_row.insert(8,'no')
       df_row_check.insert(8,'x_c2')


    #determine which team will score in next ten seconds
    if(df['team_A_scoring_within_10sec'][j] == 1):
      df_row.insert(9,'A')
      df_row_check.insert(9,'team_scoring_in_10s') 

    if(df['team_B_scoring_within_10sec'][j] == 1):
      df_row.insert(9,'B')
      df_row_check.insert(9,'team_scoring_in_10s')
    
    neither_team_scoring_wts = (df['team_A_scoring_within_10sec'][j] == 0) and (df['team_B_scoring_within_10sec'][j] == 0)

    if(neither_team_scoring_wts):
      df_row.insert(9,math.nan)
      df_row_check.insert(9,'team_scoring_in_10s')

    
  
    event_table = event_table.append({'game_num': df_row[0],
                                        'event_id' : df_row[1],
                                        'ball_loc': df_row[2],
                                        'x_a1' : df_row[3],
                                        'x_a2' : df_row[4],
                                        'x_b1' : df_row[5],
                                        'x_b2' : df_row[6],
                                        'x_c1' : df_row[7],
                                        'x_c2' : df_row[8],
                                        'team_scoring_in_10s' : df_row[9]}, ignore_index = True)


  event_tables.append(event_table)

for i in range(len(event_tables)):
  print(event_tables[i])


In [ ]:
freq_tables = dict()
for i in range(len(unique_kickoffs)):
  event_table = event_tables[i]
  row_names=["x_a1", "x_a2", "x_b1", "x_b2", "x_c1", "x_c2","team A swts", "team B swts"]
  col_names=["num_yes","num_no","p_yes","p_no"]
  data = [[1,1,1,1],[1,1,1,1],[1,1,1,1],[1,1,1,1],[1,1,1,1],[1,1,1,1],[1,1,1,1],[1,1,1,1]]
  data[0][0] = (np.array(event_table['x_a1']) == 'yes').sum()
  data[0][1] = (np.array(event_table['x_a1']) == 'no').sum()
  data[0][2] = data[0][0]/len(event_table['x_a1'])
  data[0][3] = data[0][1]/len(event_table['x_a1'])

  data[1][0] = (np.array(event_table['x_a2']) == 'yes').sum()
  data[1][1] = (np.array(event_table['x_a2']) == 'no').sum()
  data[1][2] = data[1][0]/len(event_table['x_a2'])
  data[1][3] = data[1][1]/len(event_table['x_a2'])


  data[2][0] = (np.array(event_table['x_b1']) == 'yes').sum()
  data[2][1] = (np.array(event_table['x_b1']) == 'no').sum()
  data[2][2] = data[2][0]/len(event_table['x_b1'])
  data[2][3] = data[2][1]/len(event_table['x_b1'])


  data[3][0] = (np.array(event_table['x_b2']) == 'yes').sum()
  data[3][1] = (np.array(event_table['x_b2']) == 'no').sum()
  data[3][2] = data[3][0]/len(event_table['x_b2'])
  data[3][3] = data[3][1]/len(event_table['x_b2'])


  data[4][0] = (np.array(event_table['x_c1']) == 'yes').sum()
  data[4][1] = (np.array(event_table['x_c1']) == 'no').sum()
  data[4][2] = data[4][0]/len(event_table['x_c1'])
  data[4][3] = data[4][1]/len(event_table['x_c1'])

  data[5][0] = (np.array(event_table['x_c2']) == 'yes').sum()
  data[5][1] = (np.array(event_table['x_c2']) == 'no').sum()
  data[5][2] = data[5][0]/len(event_table['x_c2'])
  data[5][3] = data[5][1]/len(event_table['x_c2'])

  data[6][0] = (np.array(event_table['team_scoring_in_10s']) == 'A').sum()
  data[6][1] = abs(data[6][0]-len(event_table['team_scoring_in_10s'])) 
  data[6][2] = data[6][0]/len(event_table['team_scoring_in_10s'])
  data[6][3] = data[6][1]/len(event_table['team_scoring_in_10s'])

  data[7][0] = (np.array(event_table['team_scoring_in_10s']) == 'B').sum()
  data[7][1] = abs([7][0]-len(event_table['team_scoring_in_10s']))
  data[7][2] = data[7][0]/len(event_table['team_scoring_in_10s'])
  data[7][3] = data[7][1]/len(event_table['team_scoring_in_10s'])

  freq_table=pd.DataFrame(data, row_names, col_names)
  event_id = unique_kickoffs[i]
  freq_tables[event_id] = (freq_table)

for i in unique_kickoffs:
  print("\nprobability table for event {}:\n{}".format(i,freq_tables[i]))

IndexError: ignored

In [ ]:
'''
compute posterior probabilities using bayes theorem P(C|X) ~ P(X|C)*P(C)
'''
#P(A-swts | x_A1, x_A2, x_B1, x_B2,x_C1, x_C2) ~ P(x_A1)*P(x_A2)*P(x_B1)*P(x_B2)*P(x_C1)*P(x_C2)*P(A-stws)

final_labels = []

n = 0

submission_df = pd.DataFrame(columns=["id","team_A_scoring_within_10sec","team_B_scoring_within_10sec"])

for i in unique_kickoffs:
  freq_table = freq_tables[i]
  pa_x = freq_table['p_yes'][0] * freq_table['p_yes'][1] * freq_table['p_yes'][2] * freq_table['p_yes'][3] \
                  * freq_table['p_yes'][4] * freq_table['p_yes'][5] * freq_table['p_yes'][6] 
  pb_x = freq_table['p_yes'][0] * freq_table['p_yes'][1] * freq_table['p_yes'][2] * freq_table['p_yes'][3] \
                  * freq_table['p_yes'][4] * freq_table['p_yes'][5] * freq_table['p_yes'][7] 

  

  final_labels.insert(n,[n,pa_x,pb_x])
  
  submission_df = submission_df.append({'id': final_labels[n][0],
                                        'team_A_scoring_within_10sec' : final_labels[n][1],
                                        'team_B_scoring_within_10sec' : final_labels[n][2]}, ignore_index = True)
  n+=1

#P(B-swts | x_A1, x_A2, x_B1, x_B2,x_C1, x_C2) ~ P(x_A1)*P(x_A2)*P(x_B1)*P(x_B2)*P(x_C1)*P(x_C2)*P(B-stws)



submission_df


,id,team_A_scoring_within_10sec,team_B_scoring_within_10sec
0,0.0,0.000000e+00,0.000000e+00
1,1.0,0.000000e+00,0.000000e+00
2,2.0,0.000000e+00,0.000000e+00
3,3.0,7.288057e-09,0.000000e+00
4,4.0,0.000000e+00,0.000000e+00
5,5.0,0.000000e+00,0.000000e+00
6,6.0,0.000000e+00,0.000000e+00
7,7.0,4.511221e-10,0.000000e+00
8,8.0,0.000000e+00,0.000000e+00
9,9.0,0.000000e+00,0.000000e+00


In [ ]:
submission_df[0]


# Everything below here is the final code. 

### I kept the above cells as a way to modularize each part, but use the code below this as the working code.

In [67]:
import numpy as np
import pandas as pd
import math
import sys

def fit(training_file):
  '''
  new approach, here i'm not using event_id or game_num as a feature
  '''
  df = pd.read_csv(training_file)

  df.pop('boost0_timer')
  df.pop('boost1_timer')
  df.pop('boost2_timer')
  df.pop('boost3_timer')
  df.pop('boost4_timer')
  df.pop('boost5_timer')

  df.pop('p0_boost')
  df.pop('p1_boost')
  df.pop('p2_boost')
  df.pop('p3_boost')
  df.pop('p4_boost')
  df.pop('p5_boost')

  df.pop('p0_pos_z')
  df.pop('p1_pos_z')
  df.pop('p2_pos_z')
  df.pop('p3_pos_z')
  df.pop('p4_pos_z')
  df.pop('p5_pos_z')

  df.pop('p0_vel_z')
  df.pop('p1_vel_z')
  df.pop('p2_vel_z')
  df.pop('p3_vel_z')
  df.pop('p4_vel_z')
  df.pop('p5_vel_z')

  df.pop('ball_pos_z')
  df.pop('ball_vel_z')

  # event x_a1 - ball closer to team A goal
  # event x_a2 - ball closer to team B goal
  # event x_b1 - ball colliding with team A player
  # event x_b2 - ball colliding with team B player
  # event x_c1 - ball rolling toward team A goal
  # event x_c2 - ball rolling toward team B goal
    




  #for each kickoff, let's consider our three events in an event table

  event_table=pd.DataFrame(columns=["ball_loc", "x_a1", "x_a2", "x_b1", "x_b2", "x_c1", "x_c2", "team_scoring_in_10s"])

  #for all snapshots in the event, we need to determine if any x_a,x_b,or x_c occurring
  for i in range(len(df)):
    

    #look through original dataframe
    
    df_row = []
    df_row_check = []
    


    #determine which side of field the ball is on
    if(df['ball_pos_x'][i] < 0): 
      df_row.insert(2,'A')
      df_row_check.insert(2,'ball_loc')

    if(df['ball_pos_x'][i] > 0): 
      df_row.insert(2,'B')
      df_row_check.insert(2,'ball_loc')

    if(df['ball_pos_x'][i] == 0): 
      df_row.insert(2,math.nan)
      df_row_check.insert(2,'ball_loc')

    min_ball_goal_dist = 5
    #is ball close to team A goal
      ##assume team A goal at x = -80 
    d_ball_goal_a = math.sqrt((df['ball_pos_x'][i]+80)**2 + df['ball_pos_y'][i]**2)

    
    
    if(d_ball_goal_a <= min_ball_goal_dist):
      df_row.insert(3,'yes')
      df_row_check.insert(3,'x_a1')

    if(d_ball_goal_a > min_ball_goal_dist):
      df_row.insert(3,'no')
      df_row_check.insert(3,'x_a1')


    #is ball close to team B goal
      ##assume team B goal at x = 80
    d_ball_goal_b = math.sqrt((df['ball_pos_x'][i]-80)**2 + df['ball_pos_y'][i]**2)
    
    if(d_ball_goal_b <= min_ball_goal_dist):
      df_row.insert(4,'yes')
      df_row_check.insert(4,'x_a2')

    if(d_ball_goal_b > min_ball_goal_dist):
      df_row.insert(4,'no')
      df_row_check.insert(4,'x_a2')




    #is ball colliding with player

    #nearest_player_idx = index of nearest player
    ##need to compute nearest player
    
    min_pbd = sys.maxsize
    min_pbi = -1
    for n in range(0,6):
      player_str = 'p{ind}'.format(ind = n)
      player_str_x = player_str +  '_pos_x'
      player_str_y = player_str +  '_pos_y'
      currdist = math.sqrt((df['ball_pos_x'][i] - df[player_str_x][i])**2 + (df['ball_pos_y'][i]-df[player_str_y][i])**2)
      if(currdist < min_pbd):
        min_pbd = currdist
        min_pbi = n

    nearest_player_idx = min_pbi
    d_player_ball = min_pbd

    min_ball_player_dist = 5

    player_on_team_a = (nearest_player_idx == 0) or (nearest_player_idx == 1) or (nearest_player_idx == 2)

    player_on_team_b = (nearest_player_idx == 3) or (nearest_player_idx == 4) or (nearest_player_idx == 5)


    #is ball colliding with team A player
    if((d_player_ball <= min_ball_player_dist) and player_on_team_a):
      df_row.insert(5,'yes')
      df_row_check.insert(5,'x_b1')
      df_row.insert(6,'no')
      df_row_check.insert(6,'x_b2')



    #is ball colliding with team B player
    if((d_player_ball <= min_ball_player_dist) and player_on_team_b):
      df_row.insert(5,'no')
      df_row_check.insert(5,'x_b1')
      df_row.insert(6,'yes')
      df_row_check.insert(6,'x_b2')


    #no player is currently colliding with ball
    if(d_player_ball > min_ball_player_dist):
      df_row.insert(5,'no')
      df_row_check.insert(5,'x_b1')
      df_row.insert(6,'no')
      df_row_check.insert(6,'x_b2')


    


    #is ball rolling toward player A goal
    if(df['ball_vel_x'][i] < 0):
        df_row.insert(7,'yes')
        df_row_check.insert(7,'x_c1')
        df_row.insert(8,'no')
        df_row_check.insert(8,'x_c2')


    #is ball rolling toward player B goal
    if(df['ball_vel_x'][i] > 0):
        df_row.insert(7,'no')
        df_row_check.insert(7,'x_c2')
        df_row.insert(8,'yes')
        df_row_check.insert(8,'x_c2')


    #ball is not moving
    if(df['ball_vel_x'][i] == 0):
        df_row.insert(7,'no')
        df_row_check.insert(7,'x_c1')
        df_row.insert(8,'no')
        df_row_check.insert(8,'x_c2')


    #determine which team will score in next ten seconds
    if(df['team_A_scoring_within_10sec'][i] == 1):
      df_row.insert(9,'A')
      df_row_check.insert(9,'team_scoring_in_10s') 

    if(df['team_B_scoring_within_10sec'][i] == 1):
      df_row.insert(9,'B')
      df_row_check.insert(9,'team_scoring_in_10s')
    
    neither_team_scoring_wts = (df['team_A_scoring_within_10sec'][i] == 0) and (df['team_B_scoring_within_10sec'][i] == 0)

    if(neither_team_scoring_wts):
      df_row.insert(9,math.nan)
      df_row_check.insert(9,'team_scoring_in_10s')

    

    event_table = event_table.append({'ball_loc': df_row[0],
                                        'x_a1' : df_row[1],
                                        'x_a2' : df_row[2],
                                        'x_b1' : df_row[3],
                                        'x_b2' : df_row[4],
                                        'x_c1' : df_row[5],
                                        'x_c2' : df_row[6],
                                        'team_scoring_in_10s' : df_row[7]}, ignore_index = True)

  row_names=["x_a1", "x_a2", "x_b1", "x_b2", "x_c1", "x_c2","team A swts", "team B swts"]
  col_names=["num_yes","num_no","p_yes","p_no"]
  data = [[1,1,1,1],[1,1,1,1],[1,1,1,1],[1,1,1,1],[1,1,1,1],[1,1,1,1],[1,1,1,1],[1,1,1,1]]
  data[0][0] = (np.array(event_table['x_a1']) == 'yes').sum()
  data[0][1] = (np.array(event_table['x_a1']) == 'no').sum()
  data[0][2] = data[0][0]/len(event_table['x_a1'])
  data[0][3] = data[0][1]/len(event_table['x_a1'])

  data[1][0] = (np.array(event_table['x_a2']) == 'yes').sum()
  data[1][1] = (np.array(event_table['x_a2']) == 'no').sum()
  data[1][2] = data[1][0]/len(event_table['x_a2'])
  data[1][3] = data[1][1]/len(event_table['x_a2'])


  data[2][0] = (np.array(event_table['x_b1']) == 'yes').sum()
  data[2][1] = (np.array(event_table['x_b1']) == 'no').sum()
  data[2][2] = data[2][0]/len(event_table['x_b1'])
  data[2][3] = data[2][1]/len(event_table['x_b1'])


  data[3][0] = (np.array(event_table['x_b2']) == 'yes').sum()
  data[3][1] = (np.array(event_table['x_b2']) == 'no').sum()
  data[3][2] = data[3][0]/len(event_table['x_b2'])
  data[3][3] = data[3][1]/len(event_table['x_b2'])


  data[4][0] = (np.array(event_table['x_c1']) == 'yes').sum()
  data[4][1] = (np.array(event_table['x_c1']) == 'no').sum()
  data[4][2] = data[4][0]/len(event_table['x_c1'])
  data[4][3] = data[4][1]/len(event_table['x_c1'])

  data[5][0] = (np.array(event_table['x_c2']) == 'yes').sum()
  data[5][1] = (np.array(event_table['x_c2']) == 'no').sum()
  data[5][2] = data[5][0]/len(event_table['x_c2'])
  data[5][3] = data[5][1]/len(event_table['x_c2'])

  data[6][0] = (np.array(event_table['team_scoring_in_10s']) == 'A').sum()
  data[6][1] = abs(data[6][0]-len(event_table['team_scoring_in_10s'])) 
  data[6][2] = data[6][0]/len(event_table['team_scoring_in_10s'])
  data[6][3] = data[6][1]/len(event_table['team_scoring_in_10s'])

  data[7][0] = (np.array(event_table['team_scoring_in_10s']) == 'B').sum()
  data[7][1] = abs([7][0]-len(event_table['team_scoring_in_10s']))
  data[7][2] = data[7][0]/len(event_table['team_scoring_in_10s'])
  data[7][3] = data[7][1]/len(event_table['team_scoring_in_10s'])

  if(data[6][2] == 0):
    data[6][2] = abs(1 - data[7][3])
    data[6][3] = abs(1 - data[6][2])

  if(data[7][2] == 0):
    data[7][2] = abs(1 - data[6][3])
    data[7][3] = abs(1 - data[6][2])

  freq_table=pd.DataFrame(data, row_names, col_names)
  freq_table['num_yes'] = freq_table['num_yes'].astype(int)
  freq_table['num_no'] = freq_table['num_no'].astype(int)
  freq_table['p_yes'] = freq_table['p_yes'].astype(float)
  freq_table['p_no'] = freq_table['p_no'].astype(float)

  return freq_table


def predict(testing_file, freq_table):
  '''
  compute posterior probabilities using bayes theorem P(C|X) ~ P(X|C)*P(C)
  '''

  test_df = pd.read_csv(testing_file)
  cov_mat = np.cov(np.array(test_df.fillna(0)).T)
  mean = compute_mean(np.array(test_df.fillna(0)))
  dim = test_df.shape[1]

  #P(A-swts | x_A1, x_A2, x_B1, x_B2,x_C1, x_C2) ~ P(x_A1)*P(x_A2)*P(x_B1)*P(x_B2)*P(x_C1)*P(x_C2)*P(A-stws)
  #P(B-swts | x_A1, x_A2, x_B1, x_B2,x_C1, x_C2) ~ P(x_A1)*P(x_A2)*P(x_B1)*P(x_B2)*P(x_C1)*P(x_C2)*P(B-stws)

  final_labels = []

  

  submission_df = pd.DataFrame(columns=["id","team_A_scoring_within_10sec","team_B_scoring_within_10sec"])

  for i in range(len(test_df)):
    p_x = p_est(np.array(test_df.iloc[i]),cov_mat,mean,dim)

    pa_x = freq_table['p_yes'][0] * freq_table['p_yes'][1] * freq_table['p_yes'][2] * freq_table['p_yes'][3] \
                    * freq_table['p_yes'][4] * freq_table['p_yes'][5] * freq_table['p_yes'][6] * p_x

    pb_x = freq_table['p_yes'][0] * freq_table['p_yes'][1] * freq_table['p_yes'][2] * freq_table['p_yes'][3] \
                    * freq_table['p_yes'][4] * freq_table['p_yes'][5] * freq_table['p_yes'][7] * p_x


    final_labels.insert(i,[i,pa_x,pb_x])

    submission_df = submission_df.append({'id': int(final_labels[i][0]),
                                          'team_A_scoring_within_10sec' : final_labels[i][1],
                                          'team_B_scoring_within_10sec' : final_labels[i][2]}, ignore_index = True)
    

  submission_df['id'] = submission_df['id'].astype(int)
  submission_df['team_A_scoring_within_10sec'] = submission_df['team_A_scoring_within_10sec'].astype(str)
  submission_df['team_B_scoring_within_10sec'] = submission_df['team_B_scoring_within_10sec'].astype(str)
  submission_df['team_A_scoring_within_10sec'] = submission_df['team_A_scoring_within_10sec'].str.slice(0,4)
  submission_df['team_B_scoring_within_10sec'] = submission_df['team_B_scoring_within_10sec'].str.slice(0,4)
  submission_df['team_A_scoring_within_10sec'] = submission_df['team_A_scoring_within_10sec'].astype(float)
  submission_df['team_B_scoring_within_10sec'] = submission_df['team_B_scoring_within_10sec'].astype(float)
  submission_df['team_A_scoring_within_10sec'] = submission_df['team_A_scoring_within_10sec']/10
  submission_df['team_B_scoring_within_10sec'] = submission_df['team_B_scoring_within_10sec']/10
  submission_df['team_A_scoring_within_10sec'] = submission_df['team_A_scoring_within_10sec'].astype(str)
  submission_df['team_B_scoring_within_10sec'] = submission_df['team_B_scoring_within_10sec'].astype(str)
  submission_df['team_A_scoring_within_10sec'] = submission_df['team_A_scoring_within_10sec'].str.slice(0,5)
  submission_df['team_B_scoring_within_10sec'] = submission_df['team_B_scoring_within_10sec'].str.slice(0,5)
  submission_df.to_csv('out.csv', index=False)  


def p_est(x, cov_mat, mean,dim):
  x_minus_m = np.zeros(dim)
  for i in range(len(x)):
    if(math.isnan(x[i])):
      x_minus_m[i] = 0
    else:
      x_minus_m[i] = float(x[i]) - float(mean[i])
  const_part = (1/((2*math.pi)**(dim/2)))*np.linalg.det(cov_mat)**(-1/2)
  result = const_part * np.exp((-1/2)*x_minus_m @ np.linalg.inv(cov_mat) @ x_minus_m.T)
  return result

def compute_mean(X):
  N,dim = X.shape
  mean = np.zeros(dim)
  for i in range(N):
    for j in range(dim):
      if(math.isnan(X[i][j])):
        mean[j] += 0
      else:
        mean[j] += float(X[i][j])
  return mean/(N-1)


def normalize(arr):
  result = []
  curr_min = sys.maxsize
  curr_max = -sys.maxsize
  for i in range(len(arr)):
    a = float(arr[i][1])
    b = float(arr[i][2])
    if(min(a,b) < curr_min):
      curr_min = min(a,b)
    if(max(a,b) > curr_max):
      curr_max = max(a,b)

  new_min = 0
  new_max = 1


  for i in range(len(arr)):
    new_a = ((a-curr_min)/(curr_max-curr_min))*(new_max-new_min)+new_min
    new_b = ((b-curr_min)/(curr_max-curr_min))*(new_max-new_min)+new_min
    result.insert(i,(arr[i][0],new_a,new_b))
  return result

freq_table = fit('train_0_truncated_tail.csv')

predict('test_truncated.csv', freq_table)


In [63]:
freq_table

,num_yes,num_no,p_yes,p_no
x_a1,5,995,0.005,0.995
x_a2,3,997,0.003,0.997
x_b1,79,921,0.079,0.921
x_b2,86,914,0.086,0.914
x_c1,468,532,0.468,0.532
x_c2,516,484,0.516,0.484
team A swts,0,1000,0.007,0.993
team B swts,91,993,0.091,0.993
